
## Indox Retrieval Augmentation


Here, we will explore how to work with Indox Retrieval Augmentation. First, if you are using OpenAI, you should set your OpenAI key as an environment variable.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/osllmai/inDox/blob/master/cookbook/indoxArcg/openai_unstructured.ipynb)

In [ ]:
!pip install openai
!pip install indoxArcg
!pip install chromadb
!pip install duckduckgo-search

## Setting Up the Python Environment

If you are running this project in your local IDE, please create a Python environment to ensure all dependencies are correctly managed. You can follow the steps below to set up a virtual environment named `indoxArcg`:

### Windows

1. **Create the virtual environment:**
```bash
python -m venv indoxArcg
```
2. **Activate the virtual environment:**
```bash
indoxArcg\Scripts\activate
```

### macOS/Linux

1. **Create the virtual environment:**
   ```bash
   python3 -m venv indoxArcg
```

2. **Activate the virtual environment:**
    ```bash
   source indoxArcg/bin/activate
```
### Install Dependencies

Once the virtual environment is activated, install the required dependencies by running:

```bash
pip install -r requirements.txt
```


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

### Creating an instance of IndoxRetrivalAugmentation

To effectively utilize the Indox Retrieval Augmentation capabilities, you must first create an instance of the IndoxRetrievalAugmentation class. This instance will allow you to access the methods and properties defined within the class, enabling the augmentation and retrieval functionalities.

### Generating response using OpenAI's language models 
OpenAIQA class is used to handle question-answering task using OpenAI's language models. This instance creates OpenAiEmbedding class to specifying embedding model. Here ChromaVectorStore handles the storage and retrieval of vector embeddings by specifying a collection name and sets up a vector store where text embeddings can be stored and queried.

In [ ]:
from indoxArcg.llms import OpenAi
from indoxArcg.embeddings import OpenAiEmbedding

openai_qa = OpenAi(api_key=OPENAI_API_KEY, model="gpt-3.5-turbo-0125")
embed_openai = OpenAiEmbedding(api_key=OPENAI_API_KEY, model="text-embedding-3-small")

from indoxArcg.vector_stores import Chroma

db = Chroma(collection_name="sample", embedding_function=embed_openai)

INFO: Initializing OpenAi with model: gpt-3.5-turbo-0125
INFO: OpenAi initialized successfully
INFO: Initialized OpenAI embeddings with model: text-embedding-3-small
INFO: Connection to the vector store database established successfully


### load and preprocess data
This part of code demonstrates how to load and preprocess text data from a file, split it into chunks, and store these chunks in the vector store that was set up previously.

In [ ]:
!wget https://raw.githubusercontent.com/osllmai/inDox/master/Demo/sample.txt

In [4]:
file_path = "sample.txt"

In [ ]:
from indoxArcg.data_loader_splitter import UnstructuredLoadAndSplit

loader_splitter = UnstructuredLoadAndSplit(file_path=file_path, max_chunk_size=400, remove_sword=False)
docs = loader_splitter.load_and_chunk()

INFO: UnstructuredLoadAndSplit initialized successfully
INFO: Getting all documents
INFO: Starting processing
INFO: Using title-based chunking
INFO: Completed chunking process
INFO: Successfully obtained all documents


In [6]:
docs[0].page_content

"The wife of a rich man fell sick, and as she felt that her endwas drawing near, she called her only daughter to her bedside andsaid, dear child, be good and pious, and then thegood God will always protect you, and I will look down on youfrom heaven and be near you. Thereupon she closed her eyes anddeparted. Every day the maiden went out to her mother's grave,"

In [7]:
db.add(docs=docs)

INFO: Storing documents in the vector store
INFO: Document added successfully to the vector store.
INFO: Documents stored successfully


### Retrieve relevant information and generate an answer
The main purpose of these lines is to perform a query on the vector store to retrieve the most relevant information (top_k=5) and generate an answer using the language model.

In [8]:
query = "How cinderella reach her happy ending?"
from indoxArcg.pipelines.rag import RAG
retriever = RAG(llm=openai_qa,vector_store=db,top_k= 5)

infer(query) method sends the query to the retriever, which searches the vector store for relevant text chunks and uses the language model to generate a response based on the retrieved information.
Context property retrieves the context or the detailed information that the retriever used to generate the answer to the query. It provides insight into how the query was answered by showing the relevant text chunks and any additional information used.

In [9]:
retriever.infer(query)

INFO: Retrieving context and scores from the vector database
INFO: Generating answer without document relevancy filter
INFO: Answering question
INFO: Generating response
INFO: Response generated successfully
INFO: Query answered successfully


'Cinderella reached her happy ending by being kind, patient, and having a pure heart. Despite facing mistreatment from her step-family, she remained humble and continued to do good deeds. With the help of a little white bird and magical assistance, Cinderella was able to attend the royal festival where the prince fell in love with her. Ultimately, her kindness, inner beauty, and resilience led her to marry the prince and live happily ever after.'